# HW2: Feature extraction based on linear independence and analysis of its impact on the loss function optimization process

Course: Computational Data Mining

Student: Mahdi Heydari | sn: 404112060

## 0: Importing libraries and packages

In [8]:
import pandas as pd
import numpy as np
from sklearn.datasets import (
    load_breast_cancer,
    fetch_california_housing, # The modern replacement for Boston Housing
    load_iris
)

## 1: Loading Datasets

### 1.1: (Classification) Wisconsin Breast Cancer

In [7]:
cancer_data = load_breast_cancer(as_frame=True)
X_class = cancer_data.data
y_class = cancer_data.target

In [53]:
print("Wisconsin Breast Cancer Dataset Summary\n")
print("# X_class info:")
print(X_class.shape[0],"samples, ", X_class.shape[1],"features")
print("features: \n", list(X_class.columns))
print("\n","-"*30)
print("\n# y_class info:")
print("Target (y): ", cancer_data.target_names[0], " (0) or ", cancer_data.target_names[1]," (1)")
print(y_class.value_counts())
# X_class.info()
# print(X_class.describe())

Wisconsin Breast Cancer Dataset Summary

# X_class info:
569 samples,  30 features
features: 
 ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']

 ------------------------------

# y_class info:
Target (y):  malignant  (0) or  benign  (1)
target
1    357
0    212
Name: count, dtype: int64


### 1.2 (Regression) Boston Housing

In [54]:
housing_data = fetch_california_housing(as_frame=True)
X_reg = housing_data.data
y_reg = housing_data.target

In [59]:
print("Boston Housing Dataset Summary\n")
print("# X_reg info:")
print(X_reg.shape[0],"samples, ", X_reg.shape[1],"features")
print("features: \n", list(X_class.columns))
print("\n","-"*30)
print("\n# y_reg info:")
print("Target (y): ", housing_data.target_names[0])
print(y_reg.describe())


Boston Housing Dataset Summary

# X_reg info:
20640 samples,  8 features
features: 
 ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']

 ------------------------------

# y_reg info:
Target (y):  MedHouseVal
count    20640.000000
mean         2.068558
std          1.153956
min          0.149990
25%          1.196000
50%          1.797000
75%          2.647250
max          5.000010
Name: MedHouseVal, dtype: float64
